<a href="https://colab.research.google.com/github/MaximR002/a-b_tests/blob/main/A%7CB_test_with_average_user_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Average user metrics - Средние метрики пользователей. Численные сигналы, например, среднее число добавленных товаров в корзину на юзера или ARPU.


##Разберем дизайн A|B теста на примере ARPU(Average revenue per user).
##Предположим, что мы придумали какую-то фичу, которая должно увеличить ARPU.
##Импортируем необходимые библиотеки, для работы с данными и статистикой

In [2]:
import numpy as np
import pandas as pd
import scipy as s

##Создадим генератор выборок

In [3]:
def generate_arpu_sample(num_users, avg_revenue_per_user, std_dev_revenue_per_user):
    """
    Генерирует выборку пользователей с заданными параметрами дохода.

    Параметры:
    num_users (int): Количество пользователей в выборке.
    avg_revenue_per_user (float): Средний доход на пользователя.
    std_dev_revenue_per_user (float): Стандартное отклонение дохода на пользователя.

    Возвращает:
    numpy.array: Массив доходов пользователей.
    """
    # Генерация выборки доходов на пользователя с нормальным распределением
    revenue_per_user = np.random.normal(avg_revenue_per_user, std_dev_revenue_per_user, num_users)

    return revenue_per_user

In [4]:
group_a = generate_arpu_sample(1000, 10, 5)
group_b = generate_arpu_sample(1000, 10, 5)

##Проведем А/А тест, для того чтобы проверить одинаковость групп.
##А также посмотрим на стандартные отклонения и выборочные средние.

In [5]:
print(f'Стандартное отклонение в контрольной группе: {group_a.std()}')
print(f'Стандартное отклонение в тестовой группе: {group_b.std()}')

Стандартное отклонение в контрольной группе: 4.885598680644524
Стандартное отклонение в тестовой группе: 5.096364075603585


In [6]:
print(f'Среднее значение в контрольной группе: {group_a.mean()}')
print(f'Среднее значение в тестовой группе: {group_b.mean()}')

Среднее значение в контрольной группе: 9.804916849551724
Среднее значение в тестовой группе: 10.032484324786964


##Будем проверять различие между группами при вероятности ошибки первого рода на уровне 0.05

In [7]:
from scipy.stats import ttest_ind

t_statistic, p_value = ttest_ind(group_a, group_b)

print("Значение t-статистики:", t_statistic)
print("p-значение:", p_value)

Значение t-статистики: -1.0188146926267951
p-значение: 0.30841429652854896


##Т.к. p-значение больше 0.05 => у нас недостаточно доказательств, чтобы отклонить нулевую гипотезу => различия в средних в групп не стат.значимы.

## Так же построим 95% - доверительный интервал, для обеих выборок, чтобы найти пересечения и удостовериться, что выборки одинаковы и мы можем использовать их для А/Б эксперимента.

In [8]:
def bootstrap_samples(count_samples, group):
  bootstrap_stat = []
  for i in range(count_samples):
    bootstrap_sample = np.random.choice(group, size=len(group), replace=True)
    bootstrap_stat.append(np.mean(bootstrap_sample))
  return np.percentile(bootstrap_stat, [2.5, 97.5])

In [9]:
print(f'Доверительныый интервал для контрольной группы: {bootstrap_samples(10000, group_a)}')
print(f'Доверительныый интервал для тестовой группы: {bootstrap_samples(10000, group_b)}')

Доверительныый интервал для контрольной группы: [ 9.50405735 10.10873851]
Доверительныый интервал для тестовой группы: [ 9.71777979 10.34238778]


## Видим, что наши доверительные интервалы накладываются => различия в группах не статистически значимы

##Сформулируем гипотезы для А/Б теста, а также обозначим ошибку 1 рода, мощность, а также mde.
###H0 - ARPU(контрольная) = ARPU(тестовая)
###H1 - ARPU(контрольная) < ARPU(тестовая)
###Ошибка 1 рода зафиксируем на уровне вероятности 0.05 (альфа)
###Ошибка 2 рода зафиксируем на уровне вероятности 0.2 (betta)
###Мощность теста будет соотвествено равна 1-0.2 = 0.8
###Эффект(mde) рассчитаем, при доступных n

In [10]:
def estimate_effect_size(std, n, alpha=0.05, power=0.8):
    """
    Расчет MDE для контроля/пилота 50/50, одинаковые std и n

    :param std: стандартной отклонение одной группы
    :param n: размер выбрки в одной группе
    :return: MDE
    """
    S = np.sqrt((std**2 / n) + (std**2 / n))
    M = s.stats.norm.ppf(q=1-alpha/2) + s.stats.norm.ppf(q=power)
    return M * S


n = 1000
std = 5
mde = estimate_effect_size(std, n)

In [11]:
print(f'Если эффект будет больше {mde}, тогда мы сможем задектировать эффект с вероятностью ошибки 1 рода и мощностью теста 0.8')

Если эффект будет больше 0.6264534992459172, тогда мы сможем задектировать эффект с вероятностью ошибки 1 рода и мощностью теста 0.8


##Предположим, что наша метрика либо увеличилась, либо уменшилась на mde*1.1.

In [12]:
# Выполняем t-тест
t_statistic, p_value = s.stats.ttest_ind(group_a, group_b-mde*1.1)

# Выводим результаты
print("Значение t-статистики:", t_statistic)
print("p-значение:", p_value)

Значение t-статистики: 2.066266035545796
p-значение: 0.03893197695195421


In [13]:
# Выполняем t-тест
t_statistic, p_value = s.stats.ttest_ind(group_a, group_b+mde*1.1)

# Выводим результаты
print("Значение t-статистики:", t_statistic)
print("p-значение:", p_value)

Значение t-статистики: -4.103895420799378
p-значение: 4.2257984364781546e-05


##И в том и другом случае мы не можем принять нулевую гипотезу о равенсте ARPU,т.к. p-value < 0.05.

In [14]:
print(f'Доверительныый интервал для контрольной группы: {bootstrap_samples(10000, group_a)}')
print(f'Доверительныый интервал для тестовой группы: {bootstrap_samples(10000, group_b+mde*1.1)}')

Доверительныый интервал для контрольной группы: [ 9.50222969 10.10461676]
Доверительныый интервал для тестовой группы: [10.40671332 11.02987072]


In [15]:
print(f'Доверительныый интервал для контрольной группы: {bootstrap_samples(10000, group_a)}')
print(f'Доверительныый интервал для тестовой группы: {bootstrap_samples(10000, group_b-mde*1.1)}')

Доверительныый интервал для контрольной группы: [ 9.50583348 10.10563556]
Доверительныый интервал для тестовой группы: [9.02872654 9.65780473]
